In [1]:
import os
import hnswlib
import numpy as np
import pandas as pd
from typing import Dict
from pathlib import Path

# Load Dataset

In [2]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])

In [3]:
dataset_dir = Path(base_dir).joinpath(f"data/dataset/{DOMAIN}/interactions")
df = pd.read_parquet(dataset_dir)
df.head()

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3092,2025-09-21 03:01:00,click,46,F,"트렌디한 스포티룩, 세미오버핏 레깅스 in 여름",아이보리,스포티,세미오버핏,울,여름,7부,레깅스
1,1,8879,2025-09-21 03:02:00,click,46,F,"트렌디한 스포티룩, 슬림핏 레깅스 in 여름",아이보리,스포티,슬림핏,메쉬,여름,롱슬리브,레깅스
2,1,1880,2025-09-21 03:03:00,click,46,F,필수템! 스포티 무드의 간절기용 레깅스,아이보리,스포티,세미오버핏,데님,간절기,숏슬리브,레깅스
3,1,6154,2025-09-21 03:13:00,click,46,F,여름 감성 슬림핏 폴리 레깅스,아이보리,스포티,슬림핏,폴리,여름,롱슬리브,레깅스
4,1,8642,2025-09-21 03:13:00,wishlist,46,F,"트렌디한 스포티룩, 레귤러핏 레깅스 in 겨울",아이보리,스포티,레귤러핏,코튼,겨울,7부,레깅스


# Load Embedding Vectors

In [4]:
save_path = Path(base_dir).joinpath(f"data/model/{DOMAIN}/lightgcn")

In [5]:
df_user_vectors = pd.read_parquet(save_path.joinpath("user_vector.parquet"))
df_item_vectors = pd.read_parquet(save_path.joinpath("item_vector.parquet"))

In [6]:
user_id_maps = dict(zip(df_user_vectors["idx"], df_user_vectors["user_id"]))
item_id_maps = dict(zip(df_item_vectors["idx"], df_item_vectors["item_id"]))

In [7]:
user_vectors = np.array(df_user_vectors["vector_normalized"].tolist())
item_vectors = np.array(df_item_vectors["vector_normalized"].tolist())

user_vectors = np.array(user_vectors).astype(np.float32)
item_vectors = np.array(item_vectors).astype(np.float32)

# Vector Similarity Search

In [8]:
def build_index(
    vectors: np.ndarray,
    *,
    space: str = "cosine",  # "ip" | "cosine" | "l2"
    M: int = 16,
    ef_construction: int = 128,
    ef: int = 32,
    num_threads: int | None = None,
) -> hnswlib.Index:
    """
    주어진 벡터로 hnswlib 인덱스를 빌드합니다.

    :param vectors: (num, dim) 형태의 float32 배열
    :param space: 유사도/거리 공간 ("ip", "cosine", "l2")
                  - "ip": 내적(점수 클수록 유사)
                  - "cosine": 코사인 거리(작을수록 유사; 필요시 후처리로 1 - d로 유사도 변환)
                  - "l2": L2 거리(작을수록 유사)
    :param M: 그래프의 연결 정도
    :param ef_construction: 빌드 시 탐색 폭
    :param ef: 검색 시 탐색 폭(리콜↑, 속도↓)
    :param num_threads: 검색에 사용할 스레드 수 (None이면 라이브러리 기본값)
    :return: hnswlib Index
    """
    if vectors.dtype != np.float32:
        vectors = vectors.astype(np.float32)

    num, dim = vectors.shape
    index = hnswlib.Index(space=space, dim=dim)
    index.init_index(max_elements=num, M=M, ef_construction=ef_construction)
    # 내부 라벨을 0..num-1로 부여
    labels = np.arange(num, dtype=np.int32)
    index.add_items(vectors, labels)
    index.set_ef(ef)
    if num_threads is not None:
        index.set_num_threads(num_threads)
    return index


def convert_to_origin_id(
    similar_indices: Dict[int, float], id_maps: Dict[int, int]
) -> Dict[int, float]:
    """
    내부 인덱스 기반 유사도 결과를 원본 ID로 매핑합니다.

    :param similar_indices: {internal_index: score}
    :param id_maps: {internal_index: original_id}
    :return: {original_id: score}
    """
    return {
        id_maps[i]: float(score) for i, score in similar_indices.items() if i in id_maps
    }


def search(
    query_vector: np.ndarray,
    index: hnswlib.Index,
    top_k: int = 5,
) -> Dict[int, float] | list[Dict[int, float]]:
    """
    쿼리 벡터(단일 또는 배치)에 대해 인덱스에서 근접 이웃을 검색합니다.
    반환 형식:
      - 단일 벡터 입력: {내부인덱스: 점수}
      - 다중 벡터 입력: [{내부인덱스: 점수}, ...] (쿼리 순서 동일)

    :param query_vector: (dim,), (1, dim) 또는 (N, dim) 형태의 float32 배열
    :param index: hnswlib 인덱스
    :param top_k: 반환할 이웃 수
    """
    # 입력 형태 정규화
    is_single = query_vector.ndim == 1
    query = query_vector.reshape(1, -1) if is_single else query_vector
    if query.dtype != np.float32:
        query = query.astype(np.float32)

    # hnswlib는 배치 질의를 지원
    labels, distances = index.knn_query(query, k=top_k)

    # 점수 해석:
    # - space=="ip": distances가 내적 점수(클수록 유사) -> 그대로 사용
    # - space=="cosine"/"l2": distances는 거리(작을수록 유사) -> 필요시 후처리에서 변환
    results: list[Dict[int, float]] = []
    for q in range(labels.shape[0]):
        result_q: Dict[int, float] = {}
        for rank, lbl in enumerate(labels[q]):
            if lbl == -1:
                continue
            score = float(distances[q][rank])
            if index.get_current_count() > 0:
                if index.space == "cosine":
                    score = 1 - score
                elif index.space == "l2":
                    score = 1 / (1 + score)
            result_q[int(lbl)] = score
        results.append(result_q)

    return results[0] if is_single else results

## User Similarity

In [9]:
# user_vectors: (num_users, dim) numpy float32 배열이라고 가정
user_index = build_index(user_vectors)
user_idx = 0
query_vector = user_vectors[user_idx]
result = search(query_vector, user_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 1.0, 9810: 0.5701902508735657, 7705: 0.5530425906181335, 556: 0.5470017194747925, 2422: 0.5410680174827576}


In [10]:
user_id = user_id_maps[user_idx]

print(f"User ID: {user_id}")
print("-----------------------------------")
for t in sorted(df[df["user_id"] == user_id]["title"].unique()):
    print(t)

User ID: 1
-----------------------------------
간절기 레귤러핏 핑크 아크릴 7부 레깅스
댄디 무드의 핑크 레깅스
댄디 무드의 화이트 청바지
여름 감성 슬림핏 폴리 레깅스
여름 감성 오버핏 폴리 청바지
여름 감성 크롭핏 아크릴 청바지
여름 시즌, 빈티지 무드의 화이트 울 민소매 슬림핏 청바지
여름 한정 린넨 소재 민트 스커트
여름에 어울리는 화이트 청바지
트렌디한 빈티지룩, 레귤러핏 청바지 in 여름
트렌디한 스포티룩, 레귤러핏 레깅스 in 겨울
트렌디한 스포티룩, 세미오버핏 레깅스 in 여름
트렌디한 스포티룩, 슬림핏 레깅스 in 여름
필수템! 빈티지 무드의 가을용 청바지
필수템! 스포티 무드의 간절기용 레깅스
필수템! 스포티 무드의 봄용 스커트


In [11]:
similar_indies = list(result.keys())[1:]
similar_user_ids = [user_id_maps[i] for i in similar_indies]

for uid in similar_user_ids:
    print(f"\nUser ID: {uid}")
    print("-----------------------------------")
    for t in sorted(df[df["user_id"] == uid]["title"]):
        print(t)


User ID: 9811
-----------------------------------
가을 루즈핏 베이지 코튼 숏슬리브 점퍼
가을 오버핏 베이지 나일론 롱슬리브 와이드팬츠
가을 오버핏 베이지 울 숏 코트
가을에 어울리는 베이지 와이드팬츠
겨울에 어울리는 베이지 점퍼
라벤더 컬러의 모던 스타일 코튼 맨투맨
모던 무드의 라벤더 맨투맨
베이지 컬러의 댄디 스타일 모달 점퍼
베이지 컬러의 시크 스타일 데님 코트
봄 감성 크롭핏 메쉬 코트
봄 시즌, 모던 무드의 라벤더 폴리 7부 세미오버핏 맨투맨
봄 오버핏 라벤더 코튼 롱 맨투맨
봄 한정 메쉬 소재 베이지 점퍼
세미오버핏 실루엣의 퍼 와이드팬츠
시크 무드의 베이지 코트
여름 슬림핏 베이지 스웨이드 롱 와이드팬츠
여름 시즌, 빈티지 무드의 화이트 울 민소매 슬림핏 청바지
여름에 어울리는 화이트 청바지
트렌디한 빈티지룩, 레귤러핏 청바지 in 여름
트렌디한 스트리트룩, 루즈핏 와이드팬츠 in 간절기
필수템! 모던 무드의 가을용 맨투맨
필수템! 빈티지 무드의 가을용 청바지

User ID: 7706
-----------------------------------
가을 루즈핏 와인 나일론 7부 스커트
간절기 세미오버핏 와인 나일론 7부 스커트
겨울 루즈핏 그레이 스웨이드 롱 레깅스
겨울 오버핏 그레이 플리스 7부 레깅스
겨울 한정 데님 소재 그레이 레깅스
겨울에 어울리는 그레이 레깅스
겨울에 어울리는 와인 스커트
댄디 무드의 화이트 청바지
브라운 컬러의 시크 스타일 레이온 카디건
시크 무드의 브라운 카디건
아이보리 컬러의 포멀 스타일 아크릴 후드티
아이보리 컬러의 포멀 스타일 퍼 후드티
여름 감성 오버핏 폴리 청바지
여름 감성 크롭핏 아크릴 청바지
여름 시즌, 시크 무드의 브라운 나일론 롱 루즈핏 카디건
필수템! 시크 무드의 겨울용 카디건

User ID: 557
-----------------------------------
가을 루즈핏 화이트 코튼 롱 조거팬츠
가을 시즌, 모던 무드의 카키 퍼 숏 오버핏 블라우스
겨

## Item Similarity

In [12]:
item_metadata_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/item_metadata.parquet"
)
df_item = pd.read_parquet(item_metadata_path)

In [13]:
# item_vectors: (num_items, dim) numpy float32 배열이라고 가정
item_index = build_index(item_vectors)
item_idx = 0
query_vector = item_vectors[item_idx]
result = search(query_vector, item_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 1.0000001192092896, 1571: 0.9909366369247437, 6420: 0.9884215593338013, 9667: 0.6018544435501099, 5493: 0.5217803716659546}


In [14]:
item_id = item_id_maps[item_idx]

print(f"Item ID: {item_id}")
print(df_item[df_item["item_id"] == item_id]["title"].item())

Item ID: 1
필수템! 스트리트 무드의 겨울용 레깅스


In [15]:
similar_indies = list(result.keys())[1:]
similar_item_ids = [item_id_maps[i] for i in similar_indies]

for item_id in similar_item_ids:
    print(f"\nItem ID: {item_id}")
    print("-----------------------------------")
    for t in sorted(df_item[df_item["item_id"] == item_id]["title"]):
        print(t)


Item ID: 1572
-----------------------------------
트렌디한 스트리트룩, 레귤러핏 레깅스 in 가을

Item ID: 6425
-----------------------------------
가을 한정 폴리 소재 그레이 레깅스

Item ID: 9673
-----------------------------------
가을 슬림핏 라벤더 레이온 숏슬리브 맨투맨

Item ID: 5496
-----------------------------------
포멀 무드의 브라운 조거팬츠


## Recommendation

In [16]:
user_idx = 0
query_vector = user_vectors[user_idx]
result = search(query_vector, item_index, top_k=10)

item_indies = list(result.keys())[1:]
recommended_item_ids = [item_id_maps[i] for i in item_indies]

recommendation = {}
for i in item_indies:
    item_id = item_id_maps[i]
    recommendation[item_id] = result[i]

recommendation

{1496: 0.7129889130592346,
 4411: 0.7112823724746704,
 7605: 0.7107353210449219,
 3730: 0.7099245190620422,
 6099: 0.7043054103851318,
 3271: 0.704046905040741,
 3092: 0.6729648113250732,
 1880: 0.6672073602676392,
 1679: 0.6614363789558411}

In [21]:
df_recommendation = df_item[df_item["item_id"].isin(recommended_item_ids)].reset_index(
    drop=True
)
df_recommendation["score"] = df_recommendation["item_id"].map(recommendation)
df_recommendation = df_recommendation.sort_values(
    "score", ascending=False, ignore_index=True
)

In [23]:
df[df["user_id"] == user_id_maps[user_idx]].reset_index(drop=True)

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3092,2025-09-21 03:01:00,click,46,F,"트렌디한 스포티룩, 세미오버핏 레깅스 in 여름",아이보리,스포티,세미오버핏,울,여름,7부,레깅스
1,1,8879,2025-09-21 03:02:00,click,46,F,"트렌디한 스포티룩, 슬림핏 레깅스 in 여름",아이보리,스포티,슬림핏,메쉬,여름,롱슬리브,레깅스
2,1,1880,2025-09-21 03:03:00,click,46,F,필수템! 스포티 무드의 간절기용 레깅스,아이보리,스포티,세미오버핏,데님,간절기,숏슬리브,레깅스
3,1,6154,2025-09-21 03:13:00,click,46,F,여름 감성 슬림핏 폴리 레깅스,아이보리,스포티,슬림핏,폴리,여름,롱슬리브,레깅스
4,1,8642,2025-09-21 03:13:00,wishlist,46,F,"트렌디한 스포티룩, 레귤러핏 레깅스 in 겨울",아이보리,스포티,레귤러핏,코튼,겨울,7부,레깅스
5,1,6099,2025-09-01 05:44:00,click,46,F,"여름 시즌, 빈티지 무드의 화이트 울 민소매 슬림핏 청바지",화이트,빈티지,슬림핏,울,여름,민소매,청바지
6,1,4411,2025-09-01 05:48:00,click,46,F,필수템! 빈티지 무드의 가을용 청바지,화이트,빈티지,루즈핏,스웨이드,가을,7부,청바지
7,1,3730,2025-09-01 05:50:00,click,46,F,"트렌디한 빈티지룩, 레귤러핏 청바지 in 여름",화이트,빈티지,레귤러핏,나일론,여름,민소매,청바지
8,1,8096,2025-09-01 05:47:00,click,46,F,여름에 어울리는 화이트 청바지,화이트,빈티지,오버핏,모달,여름,숏슬리브,청바지
9,1,2000,2025-09-18 03:59:00,click,46,F,댄디 무드의 핑크 레깅스,핑크,댄디,레귤러핏,폴리,여름,숏슬리브,레깅스


In [24]:
df_recommendation

,item_id,title,color,style,fit,material,season,sleeve,category,score
0,1496,여름 감성 오버핏 폴리 청바지,화이트,댄디,오버핏,폴리,여름,숏슬리브,청바지,0.712989
1,4411,필수템! 빈티지 무드의 가을용 청바지,화이트,빈티지,루즈핏,스웨이드,가을,7부,청바지,0.711282
2,7605,댄디 무드의 화이트 청바지,화이트,댄디,오버핏,코튼,가을,민소매,청바지,0.710735
3,3730,"트렌디한 빈티지룩, 레귤러핏 청바지 in 여름",화이트,빈티지,레귤러핏,나일론,여름,민소매,청바지,0.709925
4,6099,"여름 시즌, 빈티지 무드의 화이트 울 민소매 슬림핏 청바지",화이트,빈티지,슬림핏,울,여름,민소매,청바지,0.704305
5,3271,여름 감성 크롭핏 아크릴 청바지,화이트,댄디,크롭핏,아크릴,여름,7부,청바지,0.704047
6,3092,"트렌디한 스포티룩, 세미오버핏 레깅스 in 여름",아이보리,스포티,세미오버핏,울,여름,7부,레깅스,0.672965
7,1880,필수템! 스포티 무드의 간절기용 레깅스,아이보리,스포티,세미오버핏,데님,간절기,숏슬리브,레깅스,0.667207
8,1679,필수템! 스포티 무드의 봄용 스커트,민트,스포티,세미오버핏,메쉬,봄,숏슬리브,스커트,0.661436
